In [1]:
import pandas as pd
import datetime
from datetime import timedelta, date

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
start_date = date(2020, 1, 1)
end_date = datetime.date.today()

In [ ]:
df = pd.DataFrame()
for single_date in daterange(start_date, end_date):
    dfs = pd.read_html(f'https://www.xe.com/currencytables/?from=HKD&date={single_date.strftime("%Y-%m-%d")}')[0]
    dfs['Date'] = single_date.strftime("%Y-%m-%d")
    df = df.append(dfs)

In [ ]:
inr_df = df[df['Currency'] == 'INR']
inr_df.to_csv('hkd_data_2010_inr.csv')

In [ ]:
hist_df = pd.read_csv('hkd_data_2010_inr.csv') 

In [ ]:
start_date = date(2021, 4, 23)
end_date = datetime.date.today()

In [ ]:
df = pd.DataFrame()
for single_date in daterange(start_date, end_date):
    dfs = pd.read_html(f'https://www.xe.com/currencytables/?from=HKD&date={single_date.strftime("%Y-%m-%d")}')[0]
    dfs['Date'] = single_date.strftime("%Y-%m-%d")
    df = df.append(dfs)

In [ ]:
df.to_csv('hkd_data.csv')

In [ ]:
inr_df = df[df['Currency'] == 'INR']

In [ ]:
inr_df.pop('Rate')
inr_df.pop('Change')

In [ ]:
inr_df.head(5)

In [ ]:
inr_df = pd.concat([hist_df, inr_df], ignore_index=True)

inr_df.plot(x='Date', y='Units per HKD', figsize=(12, 8))

In [ ]:
#FB Prophet Part

import pandas as pd
from fbprophet import Prophet 
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np

In [ ]:
df= inr_df.drop(['Currency', 'Name', 'HKD per unit'], axis=1)

In [ ]:

df = df.rename(columns={'Units per HKD': 'y', 'Date': 'ds'})
#df['ds'] =  pd.to_datetime(df['ds'], format='%d/%m/%Y')
df.head(5)

In [ ]:
# to save a copy of the original data..you'll see why shortly. 
df['y_orig'] = df['y'] 
# log-transform of y
df['y'] = np.log(df['y'])

In [ ]:
#instantiate Prophet
model = Prophet() 
model.fit(df)

In [ ]:
future_data = model.make_future_dataframe(periods=10, freq = 'D')
future_data.tail()

In [ ]:
forecast_data = model.predict(future_data)
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)

In [ ]:
# make sure we save the original forecast data
forecast_data_orig = forecast_data 
forecast_data_orig['yhat'] = np.exp(forecast_data_orig['yhat'])
forecast_data_orig['yhat_lower'] = np.exp(forecast_data_orig['yhat_lower'])
forecast_data_orig['yhat_upper'] = np.exp(forecast_data_orig['yhat_upper'])
fig = model.plot(forecast_data_orig)

In [ ]:
fig2 = model.plot_components(forecast_data_orig)

In [ ]:
df['y_log']=df['y'] 
df['y']=df['y_orig']

In [ ]:
# Python
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()

In [ ]:
fig = plot_plotly(model, forecast_data_orig)  # This returns a plotly Figure
py.iplot(fig)

In [ ]:
final_df = pd.DataFrame(forecast_data_orig)
actual_chart = go.Scatter(y=df["y_orig"], name= 'Actual')
predict_chart = go.Scatter(y=final_df["yhat"], name= 'Predicted')
predict_chart_upper = go.Scatter(y=final_df["yhat_upper"], name= 'Predicted Upper')
predict_chart_lower = go.Scatter(y=final_df["yhat_lower"], name= 'Predicted Lower')
py.plot([actual_chart, predict_chart, predict_chart_upper, predict_chart_lower])